In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[2], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [12]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0,30,64
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features), activity_regularizer=l2(0.001),kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', activity_regularizer=l2(0.001),kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation= 'relu', activity_regularizer=l2(0.001),kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid', activity_regularizer=l2(0.001),kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [13]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))

In [14]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    
    
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [15]:
run_experiment(X,y)

 >#1: 94.323 
 >#2: 92.576 
 >#3: 91.266 
 >#4: 88.646 
 >#5: 94.760 
[94.3231463432312, 92.57642030715942, 91.26637578010559, 88.64628672599792, 94.75982785224915]
 Accuracy: 92.314% (+/-2.220) 


In [16]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0,30,64
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activity_regularizer=l2(0.001), kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64, kernel_size=3, activity_regularizer=l2(0.001), kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),activation='relu'))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64, kernel_size=3, activity_regularizer=l2(0.001), kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation= 'relu', activity_regularizer=l2(0.001),kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid', activity_regularizer=l2(0.001),kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [17]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_2(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [18]:
run_experiment_2(X,y)

 >#1: 61.572 
 >#2: 61.572 
 >#3: 61.572 
 >#4: 46.725 
 >#5: 50.655 
[61.57205104827881, 61.57205104827881, 61.57205104827881, 46.724891662597656, 50.65501928329468]
 Accuracy: 56.419% (+/-6.432) 


In [19]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0,30,64
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', activity_regularizer=l2(0.001), kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activity_regularizer=l2(0.001), kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation= 'relu', activity_regularizer=l2(0.001),kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(Dropout(0.25))
    model.add(Dense(50, activation= 'relu', activity_regularizer=l2(0.001),kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid', activity_regularizer=l2(0.001),kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [20]:
def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_3(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [17]:
run_experiment_3(X,y)

Epoch 1/10
29/29 [==============================] - 1s 18ms/step - loss: 6.5784 - accuracy: 0.5465
Epoch 2/10
29/29 [==============================] - 1s 18ms/step - loss: 3.9165 - accuracy: 0.7645
Epoch 3/10
29/29 [==============================] - 1s 17ms/step - loss: 2.5332 - accuracy: 0.9102
Epoch 4/10
29/29 [==============================] - 1s 18ms/step - loss: 1.9143 - accuracy: 0.9518
Epoch 5/10
29/29 [==============================] - 1s 18ms/step - loss: 1.5539 - accuracy: 0.9715
Epoch 6/10
29/29 [==============================] - 1s 18ms/step - loss: 1.3760 - accuracy: 0.9792
Epoch 7/10
29/29 [==============================] - 1s 18ms/step - loss: 1.1695 - accuracy: 0.9945
Epoch 8/10
29/29 [==============================] - 1s 18ms/step - loss: 1.1158 - accuracy: 0.9803
Epoch 9/10
29/29 [==============================] - 1s 18ms/step - loss: 1.0732 - accuracy: 0.9792
Epoch 10/10
29/29 [==============================] - 1s 17ms/step - loss: 0.9925 - accuracy: 0.9869
 >#1: 96.